Import

In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModel
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, CollectionConfig, HnswConfig, OptimizersConfig
import numpy as np

Fonction load & chunk

In [ ]:
import os

def load_text_files(directory: str):
    """
    Charge tout le contenu des fichiers .txt dans un dossier.
    """
    texts = {}
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            filepath = os.path.join(directory, filename)
            with open(filepath, "r", encoding="utf-8") as file:
                texts[filename] = file.read()
    return texts







def chunk_text_by_headers(text: str):
    """
    Découpe un texte en chunks basés sur les sections délimitées par "#".
    
    Chaque section démarre par un "#". Le texte à l'intérieur de chaque section est
    ensuite regroupé en un seul chunk, avec le caractère "#" supprimé.
    
    Args:
        text (str): Le texte à découper.
        
    Returns:
        List[str]: Liste de chunks cohérents.
    """
    chunks = []
    sections = text.split("#")
    
    for section in sections:
        if section.strip():
            chunks.append(section)
    
    return chunks







def save_chunks(chunks, output_directory, filename_base):
    """
    Sauvegarde les chunks dans des fichiers .txt séparés.
    
    Args:
        chunks (list): Liste de chunks à sauvegarder.
        output_directory (str): Dossier de destination des fichiers.
        filename_base (str): Base du nom de fichier pour chaque chunk.
    """
    os.makedirs(output_directory, exist_ok=True)
    
    for idx, chunk in enumerate(chunks):
        output_path = os.path.join(output_directory, f"{filename_base}_chunk_{idx + 1}.txt")
        with open(output_path, "w", encoding="utf-8") as file:
            file.write(chunk)







def main_load_chunk(input_directory: str, output_directory: str):
    """
    Charge les fichiers .txt, effectue le chunking et sauvegarde les chunks.
    
    Args:
        input_directory (str): Dossier contenant les fichiers .txt à traiter.
        output_directory (str): Dossier où sauvegarder les chunks générés.
    
    Returns:
        List[List[str]]: Liste de listes, chaque sous-liste contient les chunks pour un document.
    """

    texts = load_text_files(input_directory)

    all_chunks = [] 

    for filename, text in texts.items():
        print(f"Chunking du fichier : {filename}")
        chunks = chunk_text_by_headers(text)
        
        all_chunks.append(chunks)
        
        filename_base = os.path.splitext(filename)[0]
        save_chunks(chunks, output_directory, filename_base)
    
    return all_chunks


Charger et chunker les docs txt dans "documents"

In [75]:
input_directory = "documents"  # Répertoire contenant les fichiers .txt
output_directory = "chunks"    # Répertoire où sauvegarder les chunks
all_chunks = main_load_chunk(input_directory, output_directory)


Chunking du fichier : rules.txt
Chunking du fichier : faq.txt


In [88]:
all_chunks

[["Fonctionnement :\nLe joueur se déplace sur une grille à l'aide de bouton (haut, bas, gauche, droite) et résoud des problèmes mathématiques. Le but est d'atteindre une cible tout en répondant correctement aux questions mathématiques.\n\n",
  "Mécaniques de jeu :\nLe joueur commence au niveau 1. La grille de jeu s'agrandit au fur et à mesure des niveaux. Il y a différents types d'opérations mathématiques : addition, soustraction, multiplication, division, puissances, problèmes algébriques.\n\n",
  "Système de score :\nLe joueur gagne des points en répondant correctement aux questions. Le temps de réponse est pris en compte, une jauge de progression est présente. Les statistiques suivantes sont suivies : Nombre de réponses correctes, nombre total de tentatives, temps de réponse moyen, précision par type d'opération.\n\n",
  "Progression :\nLe jeu devient plus difficile au fur et à mesure que le niveau augmente. La difficulté augmente par : L'agrandissement de la grille, des opérations 

Fonction Database & Embedding

In [ ]:
def extract_subject(chunk):
    """
    Extrait le sujet d'un chunk en capturant le texte avant le premier ":".

    Cette fonction prend un texte brut (chunk) et identifie le texte situé avant 
    le premier caractère ":" pour le définir comme le sujet principal du chunk. 
    Si aucun ":" n'est trouvé, elle renvoie "Sujet inconnu".

    Args:
        chunk (str): Le texte brut du chunk.

    Returns:
        str: Le texte avant le premier ":" ou "Sujet inconnu" si ":" n'est pas présent.
    """
    if ":" in chunk:
        return chunk.split(":", 1)[0].strip()
    return "Sujet inconnu"







def load_embedding_model(model_name="intfloat/multilingual-e5-large-instruct"):
    """
    Charge un modèle d'embeddings et son tokenizer depuis Hugging Face.

    Cette fonction initialise un modèle de transformation (Transformer) 
    ainsi que son tokenizer à partir du hub Hugging Face. Le modèle est
    automatiquement déplacé sur un GPU (CUDA) s'il est disponible.

    Args:
        model_name (str, optional): Le nom du modèle Hugging Face à charger.
                                    Par défaut : "intfloat/multilingual-e5-large-instruct".

    Returns:
        tuple: Un tuple contenant le tokenizer, le modèle, et l'appareil (CPU ou GPU).
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = model.to(device)
    
    return tokenizer, model, device







def vectorize_text(texts, tokenizer, model, device):
    """
    Vectorise une liste de textes en utilisant un modèle d'embeddings.

    Cette fonction utilise le modèle et le tokenizer fournis pour transformer 
    chaque texte en un vecteur d'embedding. Les vecteurs résultants sont calculés
    en prenant la moyenne des représentations des tokens pour chaque texte.

    Args:
        texts (list of str): Une liste de textes à vectoriser.
        tokenizer (AutoTokenizer): Le tokenizer associé au modèle.
        model (AutoModel): Le modèle de transformation (Transformer) chargé.
        device (str): L'appareil sur lequel exécuter les calculs ("cuda" ou "cpu").

    Returns:
        numpy.ndarray: Un tableau numpy contenant les vecteurs d'embedding pour chaque texte.
    """
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    
    return embeddings.cpu().numpy()







def create_qdrant_collection(client, collection_name):
    """
    Crée une collection dans Qdrant si elle n'existe pas déjà.

    Cette fonction vérifie si une collection Qdrant portant le nom spécifié existe. 
    Si elle n'existe pas, elle est créée avec une configuration par défaut, 
    incluant les paramètres pour HNSW et les optimisations.

    Args:
        client (QdrantClient): Une instance de QdrantClient connectée à un serveur Qdrant.
        collection_name (str): Le nom de la collection à créer.

    Returns:
        None
    """
    try:
        client.get_collection(collection_name)
        print(f"La collection '{collection_name}' existe déjà.")
    except Exception:
        print(f"Création de la collection '{collection_name}'...")
        hnsw_config = HnswConfig(m=16, ef_construct=200, full_scan_threshold=200).model_dump()
        optimizer_config = OptimizersConfig(
            deleted_threshold=0.5,
            vacuum_min_vector_number=10000,
            default_segment_number=5,
            flush_interval_sec=30
        ).model_dump()
        vector_params = VectorParams(size=1024, distance=Distance.COSINE)

        client.create_collection(
            collection_name=collection_name,
            vectors_config=vector_params,
            hnsw_config=hnsw_config,
            optimizers_config=optimizer_config
        )
        print(f"Collection '{collection_name}' créée.")








def insert_data_to_qdrant(client, collection_name, embeddings, ids, metadata, texts):
    """
    Insère des données (embeddings, identifiants, métadonnées) dans une collection Qdrant.
    Cette fonction associe des vecteurs d'embedding à des identifiants uniques et des 
    métadonnées, puis les insère dans une collection Qdrant existante.

    Args:
        client (QdrantClient): Une instance de QdrantClient connectée à un serveur Qdrant.
        collection_name (str): Le nom de la collection dans laquelle insérer les données.
        embeddings (numpy.ndarray): Un tableau numpy contenant les vecteurs d'embedding.
        ids (list of int): Une liste d'identifiants uniques pour chaque vecteur.
        metadata (list of dict): Une liste de dictionnaires contenant les métadonnées associées.
        texts (list of str): Une liste des textes à insérer avec les métadonnées.
        
    Returns:
        None
    """
    client.upsert(
        collection_name=collection_name,
        points=[                            #liste de dictionnaires -> à bosser
            {
                "id": id_,
                "vector": embedding,
                "payload": {**meta, "texte": text} 
            }
            for id_, embedding, meta, text in zip(ids, embeddings, metadata, texts)
        ]
    )
    print(f"Les données ont été insérées dans la collection '{collection_name}'.")








def main_db_embedding(all_chunks, collection_name):
    """
    Vectorise les chunks et les insère dans une collection Qdrant avec des métadonnées.

    Cette fonction effectue les étapes suivantes :
    1. Charge le modèle d'embedding et le client Qdrant.
    2. Crée la collection Qdrant si elle n'existe pas.
    3. Génère des métadonnées pour chaque chunk, incluant :
        - La date de mise à jour.
        - La catégorie ("règles" ou "foire au question").
        - Le sujet extrait du chunk.
    4. Vectorise les chunks en utilisant le modèle d'embedding.
    5. Insère les vecteurs et les métadonnées dans Qdrant.

    Args:
        all_chunks (list of list of str): Une liste contenant deux listes de chunks :
            - La première liste correspond aux "règles".
            - La seconde liste correspond à la "foire aux questions".
        collection_name (str): Le nom de la collection Qdrant cible.

    Returns:
        None
    """
    tokenizer, model, device = load_embedding_model()
    client = QdrantClient(url="http://localhost:6333")
    create_qdrant_collection(client, collection_name)

    today_date = datetime.datetime.now().strftime("%Y-%m-%d")
    metadata = []
    texts = []

    for idx, chunks in enumerate(all_chunks):
        category = "règles" if idx == 0 else "foire au question"
        
        for chunk in chunks:
            
            subject = extract_subject(chunk)
            
            metadata.append({
                "mis à jour": today_date,
                "catégorie": category,
                "sujet": subject
            })
            texts.append(chunk)  

    embeddings = vectorize_text(texts, tokenizer, model, device)
    ids = list(range(len(texts)))

    insert_data_to_qdrant(client, collection_name, embeddings, ids, metadata, texts)


Vectoriser et envoyer les chunks dans la collection

In [87]:
collection_name = "GameRag"
main_db_embedding(all_chunks, collection_name)

Création de la collection 'GameRag'...
Collection 'GameRag' créée.
Les données ont été insérées dans la collection 'GameRag'.
